In [2]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
from trees import Node
print()

In [22]:
def meta_artistic_photo_count(s: str, x: int, y: int) -> int:
    n = len(s)
    photos = 0
    for i, c in enumerate(s):
        if not c in ('P', 'B'): continue
        d = 'P' if c == 'B' else 'B'
        # print(i)
        for a in range(i+x, min(i+y+1, n)):
            if s[a] != 'A': continue
            # print(f'  {a}')
            for j in range(a+x, min(a+y+1, n)):
                # print(f'    {j} -> {s[j]}')
                photos += int(s[j] == d)
    return photos


In [23]:
meta_artistic_photo_count('.PBAAP.B', 1, 3)

3

In [28]:
min((4-8)%10, (8-4) % 10)

4

In [43]:
def uniform_integers(a: int, b: int) -> int:
    "Count number of integers between `[a, b]` with same digits."
    la = math.floor(math.log(a, 10)) + 1
    lb = math.floor(math.log(b, 10)) + 1
    a1 = (10**la -1) // 9
    b1 = (10**lb -1) // 9
    ua = 9 * (la-1) + (a-1) // a1
    ub = 9 * (lb-1) + b // b1
    return ub - ua


In [48]:
uniform_integers(99, 999)

10

In [82]:
def artistic_photo_count(s: str, x: int, y: int) -> int:
    """
String `s` contains a photography cells including positions for:
    P - the photographer,
    A - the actor/model,
    B - the backdrop.
A valid photo can be made if the actor is placed between the
photographer and the backdrop.
An artistic photo keeps the (index) distance between each
to the inclusive interval [x, y].

Return number of artistic photos.
"""
    n = len(s)
    photos = 0
    @lru_cache(None)
    def closes(a: int, d: str) -> int:
        return sum(int(s[j] == d) for j in range(a+x, min(a+y+1, n)))
    @lru_cache(None)
    def mids(a: int, end: int, d: str) -> int:
        if a >= end: return 0
        m = mids(a+1, end, d)
        c = int(s[a] == 'A' and closes(a, d))
        #print(f'{" "*a}{d} {s[a]} {m} {c} => {m+c}')
        return m + c
    for i, c in enumerate(s):
        if not c in ('P', 'B'):
            continue
        d = 'P' if c == 'B' else 'B'
        photos += mids(i+x, min(i+y+1, n), d)
    return photos

In [83]:
artistic_photo_count('.PBAAP.B', 1, 3)

3

In [90]:
sys.setrecursionlimit(2**30)

In [177]:
def artistic_photo_count(s: str, x: int, y: int) -> int:
    """
String `s` contains a photography cells including positions for:
    P - the photographer,
    A - the actor/model,
    B - the backdrop.
A valid photo can be made if the actor is placed between the
photographer and the backdrop.
An artistic photo keeps the (index) distance between each
to the inclusive interval [x, y].

Return number of artistic photos.
"""
    w = y + 1
    ps = [0]*w + list(accumulate(map(lambda c: int(c=='P'), s)))
    ps.extend([ps[-1]]*w)
    bs = [0]*w + list(accumulate(map(lambda c: int(c=='B'), s)))
    bs.extend([bs[-1]]*w)
    actors = [i for i, c in enumerate(s, w) if c == 'A']

    pab = sum((ps[a-x] - ps[a-y-1]) * (bs[a+y] - bs[a+x-1]) for a in actors)
    bap = sum((bs[a-x] - bs[a-y-1]) * (ps[a+y] - ps[a+x-1]) for a in actors)

    return pab + bap

In [191]:
artistic_photo_count('PBAAP.B', 1, 3)

3

In [9]:
n = (0, 0)
s = set([n])

(0, 0) in s

True

In [14]:
def dijkstra(grid: List[List[int]]) -> int:
    n = len(grid)
    goal = (n-1, n-1)
    start = (0, 0)
    v = set([start])
    dc = grid[0][0]
    q = [(dc, start)]
    while q:
        d, p = heappop(q)
        if p == goal:
            return d
        for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            c = (p[0]+dx, p[1]+dy)
            if not (0 <= c[0] < n and 0 <= c[1] < n): continue
            if c in v: continue
            v.add(c)
            dc = d + grid[c[0]][c[1]]
            heappush(q, (dc, c))
    return -1


In [10]:
def a_star(grid: List[List[int]]) -> int:
    n = len(grid)
    g = n-1
    d = grid[0][0]
    q = [(d + g+g, 0, 0)]
    m = [[math.inf]*n for _ in grid]
    m[0][0] = d
    while q:
        _, px, py = heappop(q)
        d = m[px][py]
        if px == g and py == g:
            for r in m: print(r)
            return d
        for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            cx, cy = px+dx, py+dy
            if not (0 <= cx < n and 0 <= cy < n):
                continue
            dc = d + grid[cx][cy]
            if m[cx][cy] <= dc:
                continue
            h = dc + g+g -cx -cy
            heappush(q, (h, cx, cy))
            m[cx][cy] = dc
    return -1

In [16]:
a_star([[9,4,9,9], [6,7,6,4], [8,3,3,7], [7,4,9,10]])

[9, 13, 22, 31]
[15, 20, 26, 30]
[23, 23, 26, 33]
[30, 27, 35, 43]


43

In [7]:
a_star([[4,4], [3,7]])

14